In [1]:
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.sklearn.processing import SKLearnProcessor

region = boto3.session.Session().region_name

role = get_execution_role()
sklearn_processor = SKLearnProcessor(
    framework_version="0.20.0", role=role, instance_type="ml.c5.9xlarge", instance_count=1
)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/base_serializers.py:28: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 1.22.4)
  import scipy.sparse


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
import pandas as pd

input_data = "s3://kedrobucket/supply_chain_data_asset_shr/training/jane_st_forecasting/train.parquet/"
df = pd.read_parquet(input_data, filters=[("date_id", "==", 530)])
df.to_parquet("s3://kedrobucket/supply_chain_data_asset_shr/training/jane_st_forecasting/train_530.parquet")

,date_id,time_id,symbol_id,weight,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,...,responder_0,responder_1,responder_2,responder_3,responder_4,responder_5,responder_6,responder_7,responder_8,partition_id
0,530,0,1,3.839405,0.235738,-2.279464,-0.236129,0.084365,2.650537,-1.580238,...,0.056261,-0.120827,-0.116594,-0.982083,-0.064247,-0.611546,-1.388339,-0.005153,-0.460726,3
1,530,0,2,1.456004,0.525714,-1.615854,0.324606,0.684952,2.738732,-0.739372,...,-0.005006,0.135737,0.031865,0.031625,0.803434,0.446208,0.045781,0.887440,0.726905,3
2,530,0,3,0.817347,0.186738,-1.053889,-0.169176,0.334044,2.790434,-1.104299,...,0.197047,-0.028141,0.264388,-0.200726,0.525575,-0.250294,-0.393454,0.704080,-0.730494,3
3,530,0,5,2.178864,-0.118581,-2.297625,-0.374078,0.329010,2.938404,-1.603708,...,-0.054619,0.075284,-0.134378,-0.363248,0.363023,-0.635243,-0.290798,0.351972,-0.746065,3
4,530,0,7,1.514990,0.075927,-1.800633,-0.143949,0.270007,2.869976,-1.168578,...,-0.457571,-0.319509,0.099476,-1.657552,-0.839418,0.136364,-1.494249,-0.919371,0.170761,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23767,530,848,30,1.949580,-0.738439,0.174161,0.149912,0.088453,-1.069453,0.426710,...,0.276873,0.199208,1.191329,0.228478,0.081695,0.466474,0.139788,0.064130,0.348316,3
23768,530,848,33,2.106797,0.377952,-0.274560,0.046186,-0.525309,-1.012240,0.405721,...,-2.424184,-0.629675,0.091531,-0.038768,-0.018633,0.048416,0.107159,0.042012,0.210151,3
23769,530,848,34,0.840271,0.018234,-0.186046,0.494870,0.249478,-0.677065,0.755829,...,0.221095,-0.242488,0.966017,0.607620,0.329385,0.848473,0.314158,0.166702,0.354652,3
23770,530,848,36,0.464948,-0.411763,0.418005,0.052381,-0.035402,-1.042853,0.685868,...,-3.805294,-1.963969,-1.649593,-2.582088,-1.258413,-3.388226,0.071622,0.073101,0.171367,3


In [3]:
from sagemaker.processing import ProcessingInput, ProcessingOutput

sklearn_processor.run(
    code="trainer/preprocessing.py",
    inputs=[ProcessingInput(source=input_data, destination="/opt/ml/processing/input/train.parquet/")],
    outputs=[ProcessingOutput(output_name="df_530_lite", source="/opt/ml/processing/output")],
)

preprocessing_job_description = sklearn_processor.jobs[-1].describe()

output_config = preprocessing_job_description["ProcessingOutputConfig"]
for output in output_config["Outputs"]:
    if output["OutputName"] == "df_530_lite":
        preprocessed_training_data = output["S3Output"]["S3Uri"]

INFO:sagemaker:Creating processing-job with name sagemaker-scikit-learn-2024-10-22-15-13-03-267


.........['train.parquet', 'code']
Input dataframe shape: (47127338, 93)
Memory (Before):  15910.22
Imputing missing value by a forward fill, followed by a backward fill.
/miniconda3/lib/python3.7/site-packages/pandas/core/frame.py:3509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead
See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/opt/ml/processing/input/code/preprocessing.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead
See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f] = df[f].astype(np.float16)
/opt/ml/processing/input/code/preprocessing.py:13: SettingWithCopyWarning: 
A val

In [4]:
sklearn_processor.jobs[-1].describe()

{'ProcessingInputs': [{'InputName': 'input-1',
   'AppManaged': False,
   'S3Input': {'S3Uri': 's3://kedrobucket/supply_chain_data_asset_shr/training/jane_st_forecasting/train.parquet/',
    'LocalPath': '/opt/ml/processing/input/train.parquet/',
    'S3DataType': 'S3Prefix',
    'S3InputMode': 'File',
    'S3DataDistributionType': 'FullyReplicated',
    'S3CompressionType': 'None'}},
  {'InputName': 'code',
   'AppManaged': False,
   'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-574445142477/sagemaker-scikit-learn-2024-10-22-15-13-03-267/input/code/preprocessing.py',
    'LocalPath': '/opt/ml/processing/input/code',
    'S3DataType': 'S3Prefix',
    'S3InputMode': 'File',
    'S3DataDistributionType': 'FullyReplicated',
    'S3CompressionType': 'None'}}],
 'ProcessingOutputConfig': {'Outputs': [{'OutputName': 'df_530_lite',
    'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-574445142477/sagemaker-scikit-learn-2024-10-22-15-13-03-267/output/df_530_lite',
     'LocalPath': '/opt/ml/pr